# Solve the eigenvalue equation $Aq=\omega Bq$ for the Orr-Sommerfeld and Squire equations.

This notebook contains information on how we solve the Orr-Sommerfeld and squire equations using both the formulation in Schmid textbook and the primitive variable formulation.

## import libraries and plotting style

In [ ]:
# import libraries and set plotting style.  
import numpy as np
from scipy import linalg
from scipy import sparse
from scipy.special import factorial
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
#%matplotlib inline
%matplotlib notebook
plt.style.use('seaborn-notebook')

## Declare functions for Base flow (plane flow between parallel plates) and derivative operators

In [ ]:
# Declare functions for base flow and finite difference derivative operators
def base_flow(y):
    '''
    Output base flow for plane Poiseuille flow between two parallel plates
        U: U mean velocity (vector)
        Uy: dU/dy of mean belocity (vector)
        Uyy: d^2 U/dy^2 of mean velocity (scalar)
    '''
    U = 1.-y[1:-1,np.newaxis]**2 # make a n vector of shape (n,1) so U will broadcast with D2 and D4 correctly
    Uy = -2.*y[1:-1,np.newaxis] # dU/dy of base flow
    Uyy = -2. # d^2 U/dy^2 of base flow
    return U,Uy,Uyy
def get_D_Coeffs(s,d=2):
    '''
    Solve arbitrary stencil points s of length N with order of derivatives d<N
    can be obtained from equation on MIT website
    http://web.media.mit.edu/~crtaylor/calculator.html
    where the accuracy is determined as the usual form O(h^(N-d))
    
    Inputs:
        s: array like input of stencil points e.g. np.array([-3,-2,-1,0,1])
        d: order of desired derivative
    '''
    # let's solve an Ax=b problem
    N=s.size # stencil length
    A=[]
    for i in range(N):
        A.append(s**i)
    b=np.zeros(N)
    b[d] = factorial(d)
    x = np.linalg.solve(np.matrix(A),b)
    return x
def set_D(y,order=2,d=2,output_full=False):
    '''
    Input:
        y: array of y values of channel
        order: order of accuracy desired (assuming even e.g. 2,4,6,...)
        d: dth derivative
    Output:
        D: (n-2 by n) dth derivative of order O(h^order) assuming uniform y spacing
    '''
    h = y[1]-y[0] # uniform spacing
    n = y.size
    ones=np.ones(n)
    I = np.eye(n)
    # get coefficients for main diagonals
    N=order+d # how many pts needed for order of accuracy
    if N>n:
        raise ValueError('You need more points in your domain, you need %i pts and you only gave %i'%(N,n))
    Nm1=N-1 # how many pts needed if using central difference is equal to N-1
    # stencil and get Coeffs for diagonals
    s = np.arange(Nm1)-int((Nm1-1)/2) # stencil for central diff of order
    smax=s[-1] # right most stencil used (positive range)
    Coeffs = get_D_Coeffs(s,d=d)
    # loop over s and add coefficient matrices to D
    D = np.zeros_like(I)
    si = np.nditer(s,('c_index',))
    while not si.finished:
        i = si.index
        if si[0]==0:
            diag_to_add = np.diag(Coeffs[i] * ones,k=si[0])
        else:
            diag_to_add = np.diag(Coeffs[i] * ones[:-abs(si[0])],k=si[0])

        D += diag_to_add
        si.iternext()
    # alter BC so we don't go out of range on bottom of channel
    for i in range(0,smax):
        # for ith row, set proper stencil coefficients
        s = np.arange(N)-i # stencil for shifted diff of order
        Coeffs = get_D_Coeffs(s,d=d)
        D[i,:] = 0. # set row to zero
        D[i,s+i] = Coeffs # set row to have proper coefficients

        # for -ith-1 row, set proper stencil coefficients
        s = -(np.arange(N)-i) # stencil for shifted diff of order
        Coeffs = get_D_Coeffs(s,d=d)
        D[-i-1,:] = 0. # set row to zero
        D[-i-1,s-i-1] = Coeffs # set row to have proper coefficients

    if output_full:
        return (1./(h**d)) * D # do return the full matrix
    else:
        return (1./(h**d)) * D[1:-1,:] # do not return the top or bottom row

## solve using Orr-Sommerfeld Squire formulation of $\hat{v}$ and $\hat{\eta}$
The Orr-Sommerfeld and Squire equations in the wavelike results can be derived following Schmid's textbook in chapter 3 of "Stability and Transition in Shear Flows"
The equations shown here are equations (3.14) and (3.15)
$$\left[ (-i \omega+i \alpha U)(\mathcal{D}^2 - k^2) - i \alpha U'' - \frac{1}{Re} (\mathcal{D}^2 - k^2)^2 \right] \hat{v} = 0$$
$$\left[ (-i \omega+i \alpha U) - \frac{1}{Re}(\mathcal{D}^2 - k^2) \right] \hat{\eta} = -i \beta U' \hat{v}$$
And rearranging these equations to be of the form
$$A\hat{q} = \omega B\hat{q}$$
where $\hat{q} = [\hat{v}, \hat{\eta}]^\intercal$
Then the equations take the form shown here.
$$[
    (\underbrace{i \alpha Re U (-k^2) - i \alpha Re U_{yy} - k^4}_{a_A}) 
    + (\underbrace{i \alpha Re U + 2k^2}_{b_A}) \mathcal{D}^2 
    - \mathcal{D}^4 
] \hat{v} = \omega (
    \underbrace{-i Re k^2}_{a_B} 
    + \underbrace{i Re}_{b_B} \mathcal{D}^2
) \hat{v}$$
$$A_{os}\hat{v} = \omega B_{os} \hat{v}$$
Which can be solved alone for $\hat{v}$ given the boundary conditions at the walls to be $v=v'=0$

This can then be coupled with the $\hat{\eta}$ equation shown here to get the $\hat{\eta}$ modes as well.
$$[ (\underbrace{i \alpha Re U + k^2}_{c_A})
        +(\underbrace{-1}_{d_A})\mathcal{D}^2 ] \hat{\eta}
    + [
        \underbrace{i \beta Re U_y}_{f_A} ] \hat{v}
    = \omega [(\underbrace{i Re}_{c_B})] \hat{\eta}$$
$$A_{sq \eta}\hat{\eta} + A_{sq v}\hat{v} = \omega B_{sq \eta} \hat{\eta}$$
So to combine these equations to
$$A\hat{q} = \omega B\hat{q}$$
requires 
$$A = 
\begin{bmatrix}
A_{os} & 0 \\
A_{sq v} & A_{sq \eta}
\end{bmatrix}$$
and 
$$B = 
\begin{bmatrix}
B_{os} & 0 \\
B_{sq v} & B_{sq \eta}
\end{bmatrix}$$

        

In [ ]:
# functions for v and eta
def set_BCs(y,Aos,Bos,Asq_v,Bsq_v,Asq_eta,Bsq_eta,order=2,large_multiple=2.):
    '''
    Set boundary condition values to Aos, Bos, Asq_v, Bsq_v, Asq_eta, and Bsq_eta
    Modifies the (n-2 X n) matrices to be (n X n)
    Sets first/last row of Aos, Bos to have the BC v'=0 at walls
    Adds first/last rows to all matrices to have v=eta=0 at walls
    '''
    h=y[1]-y[0]
    #large_multiple=0.
    # v'=0 at walls
    # alter first and last line of Aos and Bos to contain v'=0 BC
    d=1 # derivative
    N=order+d # how many pts needed for order of accuracy
    s = np.arange(N) # stencil for shifted diff of order for i=0
    sn = -1*np.arange(N) # stencil for shifted diff of order for i=-1
    Coeffs = get_D_Coeffs(s,d=d)
    Coeffsn = get_D_Coeffs(sn,d=d)
    Aos[(0,-1),:] = 0.
    Bos[(0,-1),:] = 0.
    Aos[0,s] = Coeffs
    Bos[0,s] = large_multiple*Coeffs
    Aos[-1,sn-1] = Coeffsn
    Bos[-1,sn-1] = large_multiple*Coeffsn
    # v=0 at walls
    # rows to add to Aos and Bos at top and bottom
    zero_row = np.zeros((1,Aos[0,:].size))
    v0_at_bottom = np.copy(zero_row)
    v0_at_bottom[0,0] = 1.
    v0_at_top = np.copy(zero_row)
    v0_at_top[0,-1] = 1.
    # add them to Aos and Bos
    Aos = np.concatenate((v0_at_bottom,Aos,v0_at_top),axis=0)
    Bos = np.concatenate((large_multiple*v0_at_bottom,Bos,large_multiple*v0_at_top),axis=0)
    # eta=0 at walls
    # rows to add to Asq_v, Bsq_v, Asq_eta, Bsq_eta
    Asq_v = np.concatenate((zero_row,Asq_v,zero_row),axis=0)
    Bsq_v = np.concatenate((zero_row,Bsq_v,zero_row),axis=0)
    eta_at_bottom = v0_at_bottom
    eta_at_top = v0_at_top
    Asq_eta = np.concatenate((eta_at_bottom,Asq_eta,eta_at_top),axis=0)
    Bsq_eta = np.concatenate((large_multiple*eta_at_bottom,Bsq_eta,large_multiple*eta_at_top),axis=0)
    return (Aos,Bos,Asq_v,Bsq_v,Asq_eta,Bsq_eta)
def set_A_and_B(y,alpha=1,beta=0.,Re=2000.,compute='OSS',order=2,large_multiple=2.):
    '''
    Set A and B matrix for stated eigenvalue equation Aq=\omega Bq
    Input:
        y: array of y values of channel
        alpha=1 : alpha value for eigenvalue equation for channel flow
        Re=2000. : Reynolds number for flow
        compute: which equations to compute Orr-Sommerfeld-Squire "OSS" or "OS" or "SQ"
        order: order of accuracy of finite difference derivatives to use
    Output:
        A: matrix for LHS of eigenvalue equation
        B: matrix for RHS of eigenvalue equation
    '''
    #h=y[1]-y[0] # uniform spacing is assumed
    k2=alpha**2 + beta**2
    ialpha=1.j * alpha
    
    # identity matrix
    I = np.eye(y.size)
    # base flow
    U,Uy,Uyy = base_flow(y)
    
    # calculate derivatives
    D2 = set_D(y,order=order,d=2)
    D4 = set_D(y,order=order,d=4)
    
    # for Orr-Sommerfeld eq. of the form
    # (aA + bA*D2 - D4)*v = w*(aB+bB*D2)*v  OS
    # set aA and bA constants for each y
    aA = (ialpha*Re*(U*(-k2)-Uyy)-k2**2)
    bA = (ialpha*Re*U+2.*k2)
    # set aB and bB constants for each y
    aB = -1.j*Re*k2
    bB = 1.j*Re
    # create Aos, Bos
    Aos = -D4 + bA*D2 + aA*I[1:-1,:]
    Bos =       bB*D2 + aB*I[1:-1,:]
    
    # for Squire eq. of the form
    # (cA+dA*D2)*eta + fA*v = w*cB*eta
    # set cA,dA, and fA constants for each y
    cA = (ialpha*Re*U + k2 )
    dA = (-1.              )
    fA = (1.j*beta*Re*Uy   )
    # set cB constant for each y
    cB = (1.j*Re           )
    # create eta Asq,Bsq 
    Asq_eta = dA*D2 + cA*I[1:-1,:]
    Bsq_eta =         cB*I[1:-1,:]
    # create v Asq,Bsq
    Asq_v = fA*I[1:-1,:] # cut top and bottom of I
    Bsq_v = 0.*I[1:-1,:]
    
    # BCs
    Aos,Bos,Asq_v,Bsq_v,Asq_eta,Bsq_eta = set_BCs(y,Aos,Bos,Asq_v,Bsq_v,Asq_eta,Bsq_eta,order=order,large_multiple=large_multiple)
    
    #combine to A and B for combined eqs.
    if compute=='OSS':
        A = np.concatenate((
                np.concatenate((Aos, 0.*I),axis=1), # Orr-Sommerfeld
                np.concatenate((Asq_v,Asq_eta),axis=1)) # Squire
            ,axis=0)
        B = np.concatenate((
                np.concatenate((Bos, 0.*I),axis=1), # Orr-Sommerfeld
                np.concatenate((Bsq_v,Bsq_eta),axis=1)) # Squire
            ,axis=0)
    # if just Orr-Sommerfeld
    elif compute=='OS':
        A = Aos
        B = Bos
    elif compute=='SQ':
        A = Asq_eta
        B = Bsq_eta
    else :
        raise ValueError('compute needs to be OSS or OS instead of : %s'%compute)
        return 0
    
    return (A,B)
def set_and_solve_eig(n=201,compute='OSS',iBCs=True,iplot=True,**kwargs):
    '''
    Inputs:
        n=201:   number of pts in spatial y for finite difference scheme
        compute: which equations to compute Orr-Sommerfeld-Squire "OSS" or "OS" or "SQ"
        **kwargs: inputs for set_A_and_B and set_D and set_BCs
    Returns:
        eig:     eigenvalues of the equation
        evec:    eigenvectors of the equation
        eig_i:   index for eig and evec for max(eig) to smallest(eig) by using np.sort()
    '''
    print('inputs:')
    #print '    n=%i,  alpha=%.1f, beta=%.1f, Re=%.1e, compute=%s, order=%i'%(n,alpha,beta,Re,compute,order)
    print('    n=%i compute=%s'%(n,compute),end='')
    for k in kwargs.items():
        print(k,end='')
    print('')
    
    # create y
    y = np.linspace(-1,1,n)
    # solve eigenvalue problem
    A,B = set_A_and_B(y,compute=compute,**kwargs)#alpha=alpha,beta=beta,Re=Re,compute=compute,order=order,large_multiple=large_multiple)
    eig,evec = linalg.eig(A,b=B)
    
    # sort in order of decending eigenvalues using argsort and print max,min,nans,inf
    eig_i = eig.argsort().imag[::-1]   
    if iBCs:
        print('check max and min eig')
        print('   ',eig[eig!=np.inf][np.nanargmax(np.abs(eig[eig!=np.inf]))])
        print('   ',eig[eig!=np.inf][np.nanargmin(np.abs(eig[eig!=np.inf]))])
        print('check isnan and isinf')
        print('   ',eig[np.isnan(eig)])
        print('   ',eig[np.isinf(eig)])

        # check BCs
        print('check BCs v=v_y=eta=0')
        # check values at walls
        BadBCvn1 = evec[0,:]!=0
        BadBCvp1 = evec[n-1,:]!=0
        if compute=='OSS':
            BadBCetan1 = evec[n,:]!=0
            BadBCetap1 = evec[-1,:]!=0

        print('  bad boundary condition eigenfunctions satisfying v=0')
        print('   ',evec[0,BadBCvn1],'with associated eig-value of',eig[BadBCvn1],' v(y=-1 )=0')
        print('   ',evec[n-1,BadBCvp1],'with associated eig-value of',eig[BadBCvp1],' v(y=1 )=0')
        if compute=='OSS':
            print('  bad boundary condition eigenfunctions satisfying eta=0')
            BadBCetan1 = evec[n,:]!=0
            BadBCetap1 = evec[-1,:]!=0
            print('   ',evec[n,BadBCetan1],'with associated eig-value of',eig[BadBCetan1],' eta(y=-1)=0')
            print('   ',evec[-1,BadBCetap1],'with associated eig-value of',eig[BadBCetap1],' eta(y=1 )=0')
        # now check v' using forward and backward 1st diff 2nd order
        vy_bot = -3.*evec[0,:] + 4.*evec[1,:] - evec[2,:]
        # now v'=0 at other wall
        vy_top = 1.*evec[n-3,:] -4.*evec[n-2,:] + 3.*evec[n-1,:]
        # plot derivative at walls for every eigenfunction for v=0
        fig=plt.figure(figsize=(5,4))
        plt.title('dv/dy=0 at wall check')
        plt.plot(vy_bot[eig_i],'o',label='bot')
        plt.plot(vy_top[eig_i],'.',label='top')
        plt.ylabel(r'$\frac{d\hat{v}}{dy}(y=\pm 1)$')
        plt.xlabel(r'$\omega_{max}$ to $\omega_{min}$')
        plt.legend(loc='best',numpoints=1,frameon=False)
        fig.tight_layout()

    if iplot:
        # plot Orr-Sommerfeld-Squire spectrum
        fig=plt.figure(figsize=(4,4))
        ax=plt.subplot(111)
        ax.plot(eig.real,eig.imag,'bo')
        ax.set_xlabel(r'$\omega_r$')
        ax.set_ylabel(r'$\omega_i$')
        ax.axis([0,1,-1.,0])
        ax.set_title('eigenvalues')
        fig.tight_layout()
        plt.show()
    return eig,evec,eig_i

### Solve and reproduce Schmid textbook results using this formulation (note the difference is that here we use a finite difference formulation)

In [ ]:
# for figure 3.1a
eig,evec,eig_i = set_and_solve_eig(
    n=151,
    alpha=1.,
    beta=0.,
    Re=10000.,
    compute='OSS',
    order=4,
    large_multiple=1e7,)

In [ ]:
# for Figure 3.2b
eig,evec,eig_i = set_and_solve_eig(
    n=275, 
    alpha=0.,
    beta=1.,
    Re=10000.,
    compute='OSS',
    order=4,
    large_multiple=1.e3)

In [ ]:
# compute for figure 3.2
n=151
y=np.linspace(-1,1,n)
compute='OSS'
eig,evec,eig_i = set_and_solve_eig(
    n=n,
    alpha=1.,
    beta=1.,
    Re=5000.,
    compute=compute,
    order=2,
    large_multiple=1.e1)

#### plot A, P, and S branches

In [ ]:
# plot A,P, and S branches for figure 3.2
Ai = (0.3256-0.023j)
Pi = (0.95006-0.049325j)
Si = (0.675-0.4190j)
def add_v_y_branch(ax,n,y,eig_A_i,evec):
    ax.plot( np.abs(evec[:n,eig_A_i]),
        y,
        label=r'abs of $\omega = %.1e+%.1ej$'%(eig[eig_A_i].real,eig[eig_A_i].imag))
    ax.plot( evec[:n,eig_A_i].real,
        y,
        label=r'real of $\omega = %.1e+%.1ej$'%(eig[eig_A_i].real,eig[eig_A_i].imag))
    ax.plot( evec[:n,eig_A_i].imag,
        y,
        label=r'imag of $\omega = %.1e+%.1ej$'%(eig[eig_A_i].real,eig[eig_A_i].imag))
def add_eta_y_branch(ax,n,y,eig_A_i,evec):
    ax.plot(np.abs(evec[n:,eig_A_i]),
        y,
        label=r'abs of $\omega = %.1e+%.1ej$'%(eig[eig_A_i].real,eig[eig_A_i].imag))
    ax.plot(  evec[n:,eig_A_i].real,
        y,
        label=r'real of $\omega = %.1e+%.1ej$'%(eig[eig_A_i].real,eig[eig_A_i].imag))
    ax.plot(  evec[n:,eig_A_i].imag,
        y,
        label=r'imag of $\omega = %.1e+%.1ej$'%(eig[eig_A_i].real,eig[eig_A_i].imag))
def plot_v_eta_y_branch(n,y,eig_A_i,evec,branch_title,compute='OSS'):
    fig=plt.figure(figsize=(14,4))
    if compute=='OSS':
        ax=plt.subplot(121)
        ax2=plt.subplot(122)
    elif compute=='OS':
        ax=plt.subplot(111)
    ax.set_title(branch_title)
    add_v_y_branch(ax,n,y,eig_A_i,evec)
    if compute=='OSS':
        add_eta_y_branch(ax2,n,y,eig_A_i,evec)
        ax2.set_xlabel(r'$\hat{\eta}$')
        ax2.set_ylabel(r'$y$')
        ax2.legend(loc='upper left',frameon=False,numpoints=1,bbox_to_anchor=(0.95,1.05),)
    ax.set_xlabel(r'$\hat{v}$')
    ax.set_ylabel(r'$y$')
    fig.tight_layout()
    plt.subplots_adjust(left=None, right=0.6, top=None, bottom=None)

    
def Plot_APS(n,y,compute,Ai,Pi,Si,eig,evec):
    '''
    Plot the A, P, and S eigenvectors (three given complex points)
    Output, plots similar to those in Schmid
    '''
    eig_A_i = (np.abs(eig-Ai)).argmin()
    eig_P_i = (np.abs(eig-Pi)).argmin()
    eig_S_i = (np.abs(eig-Si)).argmin()

    #eig_S_i = (np.abs(eig-(0.6735-0.458))).argmin()
    # A,P, and S branches
    plot_v_eta_y_branch(n,y,eig_A_i,evec,'A branch',compute=compute)
    plot_v_eta_y_branch(n,y,eig_P_i,evec,'P branch',compute=compute)
    plot_v_eta_y_branch(n,y,eig_S_i,evec,'S branch',compute=compute)

    plt.show()
Plot_APS(n,y,compute,Ai,Pi,Si,eig,evec)

#### Compare in Plane Poiseuille to Schmid table for $\alpha=1, \beta=0, Re=2000$

In [ ]:
# mesh convergence
cos=np.array([
    0.31210030-0.01979866j,
    0.42418427-0.07671992j,
    0.92078667-0.07804706j,
    0.92091806-0.07820060j,
    0.85717055-0.13990151j,
    0.85758968-0.14031674j,
    0.79399812-0.20190508j,
    0.79413424-0.20232063j,
    0.63912513-0.22134137j,
    0.53442105-0.22356175j,
])
csq=np.array([
    0.98418861-0.01631139j,
    0.95256584-0.04793417j,
    0.92094306-0.07955694j,
    0.88932028-0.11117972j,
    0.24936056-0.13725811j,
    0.24936056-0.13725811j,
    0.85769752-0.14280249j,
    0.82607494-0.17442537j,
    0.79445264-0.20605114j,
    0.42863639-0.22466515j,
])
# combine Schmid
c=np.concatenate((cos,csq))
c_i = c.imag.argsort()[::-1]
def RSSE_n(order=2,iplot=False,n_all=[51,61,71,81,91,101,151],large_multiple=0.):
    RSSE_n=[]
    for n in n_all:
        y=np.linspace(-1,1,n)
        compute='OSS'
        # solve eigenvalue problem
        A,B = set_A_and_B(y,alpha=1.,beta=0.,Re=2000.,compute='OSS',order=order,large_multiple=0.)
        eig,evec = linalg.eig(A,b=B)
        # reorder in complex values first
        eig_i = eig.imag.argsort()[::-1]   

        # plot against table
        if iplot:
            fig=plt.figure() 
            ax=plt.subplot(111)
            ax.plot(cos.real,cos.imag,'s',label='cos Schmid')
            ax.plot(csq.real,csq.imag,'s',label='csq Schmid')
            ax.plot(eig[eig_i][6:27].real,eig[eig_i][6:27].imag,'.',label='Finit Difference')
            ax.set_xlabel(r'$\omega_r$')
            ax.set_ylabel(r'$\omega_i$')
            ax.legend(loc='best',numpoints=1)
            fig.tight_layout()
            plt.show()

        # calc error as root sum squared
        RSSE=0
        for ci in c:
            min_error = (np.abs(eig[eig_i]-ci).min())**2 # minimum square value
            RSSE+=min_error # sum
        RSSE=np.sqrt(RSSE) # root
        RSSE_n.append(RSSE)
        print('for n = %i, order = %i, then RSSE = %.3e'%(n,order,RSSE))
    return (eig,eig_i,n_all,RSSE_n)

def calc_and_plot_eigs_RSSE(axdiff=None,axcon=None,order=2,n_all=[51,61,71,81,91,101,151],large_multiple=0.):
    # calc eigenvalues of order
    eig,eig_i,n_all,RSSE_n2 = RSSE_n(order=order,n_all=n_all,large_multiple=large_multiple)
    # plot eigenvalues
    axdiff.plot(eig[eig_i][5:26].real,eig[eig_i][5:26].imag,'.',label=r'Finite Difference $O(h^{%i})$'%order)
    # plot error convergence
    axcon.loglog(n_all,RSSE_n2,'o-',label=r'Finite Difference $O(h^{%i})$'%order)
    
fig1=plt.figure(figsize=(4,4))  # plot for finite diffs
axdiff=plt.subplot(111)
axdiff.plot(cos.real,cos.imag,'s',label='cos Schmid')
axdiff.plot(csq.real,csq.imag,'s',label='csq Schmid')
axdiff.set_xlabel(r'$\omega_r$')
axdiff.set_ylabel(r'$\omega_i$')

fig2=plt.figure(figsize=(4,4)) 
axcon=plt.subplot(111)
axcon.loglog([90,180],[3.547e-4,3.547e-4/2**10],'b',label=r'$O(h^{%i})$ line'%10)
axcon.loglog([90,180],[3.823e-4,3.823e-4/2**8],'y',label=r'$O(h^%i)$ line'%8)
axcon.loglog([90,180],[1.010e-3,1.010e-3/2**6],'g',label=r'$O(h^%i)$ line'%6)
axcon.loglog([90,180],[2.79e-3,2.79e-3/2**4],'r',label=r'$O(h^%i)$ line'%4)
axcon.loglog([90,180],[1.56e-2,1.56e-2/2**2],'m',label=r'$O(h^%i)$ line'%2)
axcon.set_xlabel(r'$n$')
axcon.set_ylabel(r'$RSSE$')

#n_all = [81,91,101,111,121,162]
n_all = [101,161,201]
calc_and_plot_eigs_RSSE(axdiff=axdiff,axcon=axcon,order=10,n_all=n_all,large_multiple=1.e3)
calc_and_plot_eigs_RSSE(axdiff=axdiff,axcon=axcon,order=8,n_all=n_all,large_multiple=1.e3)
calc_and_plot_eigs_RSSE(axdiff=axdiff,axcon=axcon,order=6,n_all=n_all,large_multiple=1.e3)
calc_and_plot_eigs_RSSE(axdiff=axdiff,axcon=axcon,order=4,n_all=n_all,large_multiple=1.e3)
calc_and_plot_eigs_RSSE(axdiff=axdiff,axcon=axcon,order=2,n_all=[141,201],large_multiple=1.e3)
axdiff.legend(loc='best',numpoints=1,frameon=False)
axcon.legend(loc='best',numpoints=1,frameon=False)
fig1.tight_layout()
fig2.tight_layout()
plt.show()

### Now compute $\hat{P}$ from $\hat{v}$
using the Pressure relation equation
$$\nabla^2 P = -2 U_y v_x$$
which yields the wavelike equations
$$(\mathcal{D}^2 - k^2)\hat{P} = -2 U_y i \alpha \hat{v}$$
where $\mathcal{D}^2$ is the second derivative w.r.t the $y$ axis

Since $\hat{v}$ is known, this becomes a $Ax=b$ linear problem when the boundary conditions are specified at the wall.  We shall say $\hat{P}(y=\textrm{wall}) = 0$ and that $x=\hat{P}$

In [ ]:
# solve the equations for P
# solve for eig,evec of v and eta first
n=151
y=np.linspace(-1,1,n)
alpha=1.
beta=1.
eig,evec,eig_i = set_and_solve_eig(
    n=n,
    iplot=True,
    iBCs=False,
    alpha=alpha,
    beta=beta,
    Re=5000.,
    compute='OSS',
    order=2,
    large_multiple=1e7,)

# get A,P, and S branch eigen indexes
Ai = (0.3256-0.023j)
Pi = (0.95006-0.049325j)
Si = (0.675-0.4190j)

# use v to get P by setting up Ax=b as described above
def calc_P_from_v_eta(y,n,alpha,beta,eig,evec,eig_i):
    k2 = alpha**2 + beta**2
    D2 = set_D(y,order=2,d=2)
    A = D2 - k2*np.eye(n)[1:-1,:]
    # rows to add to Aos and Bos at top and bottom
    zero_row = np.zeros((1,A[0,:].size))
    P0_at_bottom = np.copy(zero_row)
    P0_at_bottom[0,0] = 1.
    P0_at_top = np.copy(zero_row)
    P0_at_top[0,-1] = 1.
    A = np.concatenate((P0_at_bottom,A,P0_at_top),axis=0)

    # set b matrix from v's
    U,Uy,Uyy = base_flow(y)
    b = -2.*Uy*1.j*alpha*evec[1:n-1,:] # skip wall equations
    # add BC equations
    b = np.concatenate((np.zeros((1,2*n)),b,np.zeros((1,2*n))),axis=0)
    P = np.linalg.solve(A,b)
    return P
P = calc_P_from_v_eta(y,n,alpha,beta,eig,evec,eig_i)

def add_P_y_branch(ax,n,y,eig_Ai,P,branch_title):
    ax.set_title(branch_title + r' $\omega = %.1e+%.1ej$'%(eig[eig_Ai].real,eig[eig_Ai].imag))
    ax.plot( np.abs(P[:,eig_Ai]),
        y,
        label=r'abs')
    ax.plot( P[:,eig_Ai].real,
        y,
        label=r'real')
    ax.plot( P[:,eig_Ai].imag,
        y,
        label=r'imag')
def plot_P_all_branches(y,n,Ai,Pi,Si,P,title=''):
    # get indices of branches
    eig_A_i = (np.abs(eig-Ai)).argmin()
    eig_P_i = (np.abs(eig-Pi)).argmin()
    eig_S_i = (np.abs(eig-Si)).argmin()
    # open figure and plot the three branches
    fig=plt.figure(figsize=(3,7))
    ax=plt.subplot(311)
    ax2=plt.subplot(312)
    ax3=plt.subplot(313)
    add_P_y_branch(ax,n,y,eig_A_i,P,title+' A')
    add_P_y_branch(ax2,n,y,eig_P_i,P,title+' P')
    add_P_y_branch(ax3,n,y,eig_S_i,P,title+' S')
    ax.legend(loc='best',frameon=True,numpoints=1)
    ax2.legend(loc='best',frameon=False,numpoints=1)
    ax3.legend(loc='best',frameon=False,numpoints=1)
    fig.tight_layout()
    plt.show()
plot_P_all_branches(y,n,Ai,Pi,Si,P)


### Now compute $\hat{u}$, $\hat{w}$ from the $\eta$ equation and continuity
The $\eta$ equation is shown here:
$$\eta = \frac{\partial u}{\partial z} - \frac{\partial w}{\partial x}$$
This can have a wavelike solution introduced as well, which will yield
$$i \beta \hat{u} - i \alpha \hat{w} = \hat{\eta}$$
Also, we can use continuity $\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} + \frac{\partial w}{\partial z} = 0$ which can have a wavelike solution introduced to yield:
$$i \alpha \hat{u} + i \beta \hat{w} = -\mathcal{D}^1 \hat{v}$$
We can combine these equation to a linear system and solve for $x$
$$ A x = b$$
where $x=[\hat{u}, \hat{w}]^\intercal$


In [ ]:
# calculate u,w, from v and eta
def calc_u_w_from_v_eta(y,n,alpha,beta,eig,evec,eig_i):
    k2 = alpha**2+beta**2
    D1 = set_D(y,order=2,d=1)/(y[1]-y[0])
    dvdy = np.gradient(evec[:n,:],axis=0)[1:-1,:]
    U,Uy,Uyy = base_flow(y)
    
    # lets do eta equation first
    Aueta = (1.j*alpha)*np.eye(n)[1:-1,:]
    Aweta = (-1.j*beta)*np.eye(n)[1:-1,:]
    # rows to add to Au and Aw at top and bottom
    zero_row = np.zeros((1,Aueta[0,:].size))
    uw0_at_bottom = np.copy(zero_row)
    uw0_at_bottom[0,0] = 1.
    uw0_at_top = np.copy(zero_row)
    uw0_at_top[0,-1] = 1.
    Aueta = np.concatenate((uw0_at_bottom,Aueta,uw0_at_top),axis=0)
    Aweta = np.concatenate((zero_row,Aweta,zero_row),axis=0)
    # set b matrix from eta's
    b_eta = evec[n+1:-1,:] # skip wall equations
    # add BC equations
    b_eta = np.concatenate((np.zeros((1,2*n)),b_eta,np.zeros((1,2*n))),axis=0)
    
    # now lets do continuity equation
    Aucon = (1.j*alpha)*np.eye(n)[1:-1,:]
    Awcon = (1.j*beta)*np.eye(n)[1:-1,:]
    # rows to add to Au and Aw at top and bottom
    zero_row = np.zeros((1,Aucon[0,:].size))
    uw0_at_bottom = np.copy(zero_row)
    uw0_at_bottom[0,0] = 1.
    uw0_at_top = np.copy(zero_row)
    uw0_at_top[0,-1] = 1.
    Aucon = np.concatenate((zero_row,Aucon,zero_row),axis=0)
    Awcon = np.concatenate((uw0_at_bottom,Awcon,uw0_at_top),axis=0)
    # set b matrix from v's
    bcon = -dvdy # skip wall equations
    # add BC equations
    bcon = np.concatenate((np.zeros((1,2*n)),bcon,np.zeros((1,2*n))),axis=0)
    
    A = np.concatenate((
                np.concatenate((Aueta, Aweta),axis=1), # eta
                np.concatenate((Aucon, Awcon),axis=1)) # continuity
            ,axis=0) 
    b =  np.concatenate((b_eta,         # eta
                         bcon),axis=0)  # continuity
    
    uw = np.linalg.solve(A,b)
    return uw
uw = calc_u_w_from_v_eta(y,n,alpha,beta,eig,evec,eig_i)
plot_P_all_branches(y,n,Ai,Pi,Si,uw[:n,:],title='u')
plot_P_all_branches(y,n,Ai,Pi,Si,uw[n:,:],title='w')

## Solve u,v,w, and P using primitive variables
Formulation is shown here from the momentum and continuity equations:
$$\frac{\partial u}{\partial t} + U \frac{\partial u}{\partial x} + v U_y = -\frac{\partial P}{\partial x} + \frac{1}{Re} \nabla^2 u$$
$$\frac{\partial v}{\partial t} + U \frac{\partial v}{\partial x} = -\frac{\partial P}{\partial y} + \frac{1}{Re} \nabla^2 v$$
$$\frac{\partial w}{\partial t} + U \frac{\partial w}{\partial x} = -\frac{\partial P}{\partial z} + \frac{1}{Re} \nabla^2 w$$
$$\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} + \frac{\partial w}{\partial z} = 0$$
Then, introducing the wavelike solution and rearranging we find these equations become
$$(
    i \alpha Re U + k^2 
    - \mathcal{D}^2
  )\hat{u}
  + Re U_y \hat{v}
  + Re i \alpha \hat{P} 
  = \omega i Re \hat{u}$$
$$( 
    i \alpha Re U + k^2
    - \mathcal{D}^2
  )\hat{v} 
  + Re \mathcal{D} \hat{P}
  = \omega i Re \hat{v}$$
$$(
    i \alpha Re U + k^2
    - \mathcal{D}^2
    )\hat{w}
    + i Re \hat{P} = \omega i Re \hat{w}$$
$$i \alpha \hat{u} + \mathcal{D} \hat{v} + i \beta \hat{w} = 0$$
Then, we can set the BCs of $\hat{u}=\hat{v}=\hat{w}=0$ at the walls.  This gives the 6 needed BC's for the problem.  


In [ ]:
# functions for primitive formulation u,v,w,P
def set_BCs_primitive(y, Aus, Bus, Avs, Bvs, Aws, Bws, Acons, Bcons, order=2,large_multiple=0.):
    '''
    Set boundary condition values to As, Bs
    Modifies the (n-2 X n) matrices to be (n X n)
    Sets first/last row of Aos, Bos to have the BC v'=0 at walls
    Adds first/last rows to all matrices to have v=eta=0 at walls
    
    Inputs:
        y: y spactial array 
        Aus: List of [Auu, Auv, Auw, Aup]
        Bus: List of [Buu, Buv, Buw, Bup]
        Avs: List of [Avu, Avv, Avw, Avp]
        Bvs: List of [Bvu, Bvv, Bvw, Bvp]
        Aws: List of [Awu, Awv, Aww, Awp]
        Bws: List of [Bwu, Bwv, Bww, Bwp]
        Acons: List of [Aconu, Aconv, Aconw, Aconp]
        Bcons: List of [Bconu, Bconv, Bconw, Bconp]
        order: order of derivative (O(h^order))
        large_multiple: used to set RHS to nonzero and kill spurious eigenmodes
    '''
    h=y[1]-y[0]
    # v'=0 at walls
    # alter first and last line of As and Bs to contain u'=v'=w'=0 BC
    d=1 # derivative
    N=order+d # how many pts needed for order of accuracy
    s = np.arange(N) # stencil for shifted diff of order for i=0
    sn = -1*np.arange(N) # stencil for shifted diff of order for i=-1
    Coeffs = get_D_Coeffs(s,d=d)
    Coeffsn = get_D_Coeffs(sn,d=d)
    # set u-mom u'=0 at walls
    #for i in range(len(Aus)):
    #    Aus[i][(0,-1),:] = 0.
    #    Bus[i][(0,-1),:] = 0.
    #    if i==0:
    #        Aus[i][0,s] = Coeffs
    #        Bus[i][0,s] = large_multiple*Coeffs
    #        Aus[i][-1,sn-1] = Coeffsn
    #        Bus[i][-1,sn-1] = large_multiple*Coeffsn
    ## set v-mom v'=0 at walls
    #for i in range(len(Avs)):
    #    Avs[i][(0,-1),:] = 0.
    #    Bvs[i][(0,-1),:] = 0.
    #    if i==1:
    #        Avs[i][0,s] = Coeffs
    #        Bvs[i][0,s] = large_multiple*Coeffs
    #        Avs[i][-1,sn-1] = Coeffsn
    #        Bvs[i][-1,sn-1] = large_multiple*Coeffsn
    #        
    ## set w-mom w'=0 at walls
    #for i in range(len(Aws)):
    #    Aws[i][(0,-1),:] = 0.
    #    Bws[i][(0,-1),:] = 0.
    #    if i==2:
    #        Aws[i][0,s] = Coeffs
    #        Bws[i][0,s] = large_multiple*Coeffs
    #        Aws[i][-1,sn-1] = Coeffsn
    #        Bws[i][-1,sn-1] = large_multiple*Coeffsn
            
    # v=0 at walls
    # rows to add to As and Bs at top and bottom
    zero_row = np.zeros((1,Aus[0][0,:].size))
    v0_at_bottom = np.copy(zero_row)
    v0_at_bottom[0,0] = 1.
    v0_at_top = np.copy(zero_row)
    v0_at_top[0,-1] = 1.
    # add them to As and Bs
    # for Aus Bus u=0
    for i in range(len(Aus)):
        if i==0:
            Aus[i] = np.concatenate((v0_at_bottom,Aus[i],v0_at_top),axis=0)
            Bus[i] = np.concatenate((large_multiple*v0_at_bottom,Bus[i],large_multiple*v0_at_top),axis=0)
        else:
            Aus[i] = np.concatenate((zero_row,Aus[i],zero_row),axis=0)
            Bus[i] = np.concatenate((zero_row,Bus[i],zero_row),axis=0)
    # for Avs Bvs v=0
    for i in range(len(Avs)):
        if i==1:
            Avs[i] = np.concatenate((v0_at_bottom,Avs[i],v0_at_top),axis=0)
            Bvs[i] = np.concatenate((large_multiple*v0_at_bottom,Bvs[i],large_multiple*v0_at_top),axis=0)
        else:
            Avs[i] = np.concatenate((zero_row,Avs[i],zero_row),axis=0)
            Bvs[i] = np.concatenate((zero_row,Bvs[i],zero_row),axis=0)
    # for Aws Bws w=0
    for i in range(len(Aws)):
        if i==2:
            Aws[i] = np.concatenate((v0_at_bottom,Aws[i],v0_at_top),axis=0)
            Bws[i] = np.concatenate((large_multiple*v0_at_bottom,Bws[i],large_multiple*v0_at_top),axis=0)
        else:
            Aws[i] = np.concatenate((zero_row,Aws[i],zero_row),axis=0)
            Bws[i] = np.concatenate((zero_row,Bws[i],zero_row),axis=0)
    # for Acons Bcons P=0
    #for i in range(len(Acons)):
    #    if i==3:
    #        Acons[i] = np.concatenate((v0_at_bottom,Acons[i],v0_at_top),axis=0)
    #        Bcons[i] = np.concatenate((large_multiple*v0_at_bottom,Bcons[i],large_multiple*v0_at_top),axis=0)
    #    else:
    #        Acons[i] = np.concatenate((zero_row,Acons[i],zero_row),axis=0)
    #        Bcons[i] = np.concatenate((zero_row,Bcons[i],zero_row),axis=0)
    return (Aus,Bus,Avs, Bvs, Aws, Bws, Acons, Bcons)
def set_A_and_B_primitive(y,alpha=1,beta=0.,Re=2000.,order=2,large_multiple=2.):
    '''
    Set A and B matrix for stated eigenvalue equation Aq=\omega Bq
    Input:
        y: array of y values of channel
        alpha=1 : alpha value for eigenvalue equation for channel flow
        Re=2000. : Reynolds number for flow
        order: order of accuracy of finite difference derivatives to use
    Output:
        A: matrix for LHS of eigenvalue equation
        B: matrix for RHS of eigenvalue equation
    '''
    #h=y[1]-y[0] # uniform spacing is assumed
    k2=alpha**2 + beta**2
    ialpha=1.j * alpha
    ibeta =1.j * beta
    
    # identity matrix
    I = np.eye(y.size)
    Im2=I[1:-1,:] # skip first and last rows (at walls)
    zero_mat=0.*Im2
    zero_mat_full = 0.*I
    # base flow
    U,Uy,Uyy = base_flow(y)
    
    # calculate derivatives
    D1 = set_D(y,order=order,d=1)
    D1_full = set_D(y,order=order,d=1,output_full=True)
    D2 = set_D(y,order=order,d=2)
    
    # for Orr-Sommerfeld eq. of the form
    # set constants and Au Bu
    au = (ialpha*Re*U + k2)
    bu = -1.
    cu = Uy * Re
    du = ialpha * Re
    fu = 1.j * Re
    Auu = au*Im2 + bu*D2
    Auv = cu*Im2
    Auw = zero_mat
    Aup = du*Im2
    Aus = [Auu, Auv, Auw, Aup]
    Buu = fu*Im2
    Buv = zero_mat
    Buw = zero_mat
    Bup = zero_mat
    Bus = [Buu, Buv, Buw, Bup]
    
    # for constants and Av Bv
    av = (ialpha*Re*U + k2)
    bv = -1.
    cv = 0.
    dv = Re
    fv = 1.j * Re
    Avu = zero_mat
    Avv = av*Im2 + bv*D2
    Avw = zero_mat
    Avp = dv*D1
    Avs = [Avu, Avv, Avw, Avp]
    Bvu = zero_mat
    Bvv = fv*Im2
    Bvw = zero_mat
    Bvp = zero_mat
    Bvs = [Bvu, Bvv, Bvw, Bvp]
    
    # for constants and Aw Bw
    aw = (ialpha*Re*U + k2)
    bw = -1.
    cw = 0.
    dw = ibeta * Re
    fw = 1.j * Re
    Awu = zero_mat
    Awv = zero_mat
    Aww = aw*Im2 + bw*D2
    Awp = dw*Im2
    Aws = [Awu, Awv, Aww, Awp]
    Bwu = zero_mat
    Bwv = zero_mat
    Bww = fw*Im2
    Bwp = zero_mat
    Bws = [Bwu, Bwv, Bww, Bwp]
    
    # for constants and Acon Bcon
    acon = ialpha
    bcon = 1.
    ccon = ibeta
    Aconu = acon*I #do full array, since no BCs to add
    Aconv = bcon*D1_full
    Aconw = ccon*I
    Aconp = zero_mat_full
    Acons = [Aconu, Aconv, Aconw, Aconp]
    Bconu = zero_mat_full
    Bconv = zero_mat_full
    Bconw = zero_mat_full
    Bconp = zero_mat_full
    Bcons = [Bconu, Bconv, Bconw, Bconp]
    
    # BCs
    Aus,Bus,Avs,Bvs,Aws,Bws,Acons,Bcons = set_BCs_primitive(y, Aus, Bus, Avs, Bvs, Aws, Bws, Acons, Bcons, order=order,large_multiple=large_multiple)
    
    #combine to A and B for combined eqs.
    A = np.concatenate((
            np.concatenate(Aus,axis=1), # u-mom
            np.concatenate(Avs,axis=1), # v-mom
            np.concatenate(Aws,axis=1), # w-mom
            np.concatenate(Acons,axis=1))# continuity
        ,axis=0)
    B = np.concatenate((
            np.concatenate(Bus,axis=1), # u-mom
            np.concatenate(Bvs,axis=1), # v-mom
            np.concatenate(Bws,axis=1), # w-mom
            np.concatenate(Bcons,axis=1)) # continuity
        ,axis=0)
    
    return (A,B)
def set_and_solve_eig_primitive(n=201,iBCs=True,iplot=True,**kwargs):
    '''
    Inputs:
        n=201:   number of pts in spatial y for finite difference scheme
        **kwargs: inputs for set_A_and_B and set_D and set_BCs
    Returns:
        eig:     eigenvalues of the equation
        evec:    eigenvectors of the equation
        eig_i:   index for eig and evec for max(eig) to smallest(eig) by using np.sort()
    '''
    print('inputs:')
    print('    n=%i'%(n),end='')
    for k in kwargs.items():
        print(k,end='')
    print('')
    
    # create y
    y = np.linspace(-1,1,n)
    # solve eigenvalue problem
    A,B = set_A_and_B_primitive(y,**kwargs)
    eig,evec = linalg.eig(A,b=B)
    
    # sort in order of decending eigenvalues using argsort and print max,min,nans,inf
    eig_i = eig.argsort().imag[::-1]   
    if iBCs:
        print( 'check max and min eig')
        print( '   ',eig[eig!=np.inf][np.nanargmax(np.abs(eig[eig!=np.inf]))])
        print( '   ',eig[eig!=np.inf][np.nanargmin(np.abs(eig[eig!=np.inf]))])
        print( 'check isnan and isinf')
        print( '   ',eig[np.isnan(eig)])
        print( '   ',eig[np.isinf(eig)])

        # check BCs
        print('check BCs u=u_y=v=v_y=w=w_y=0')
        # check values at walls
        BadBCun1 = evec[0,:]!=0
        BadBCup1 = evec[n-1,:]!=0
        BadBCvn1 = evec[n,:]!=0
        BadBCvp1 = evec[2*n-1,:]!=0
        BadBCwn1 = evec[2*n,:]!=0
        BadBCwp1 = evec[3*n-1,:]!=0
        #BadBCpn1 = evec[3*n,:]!=0
        #BadBCpp1 = evec[4*n-1,:]!=0

        print( '  bad boundary condition eigenfunctions satisfying u=0')
        print( '   ',evec[0,BadBCun1],'with associated eig-value of',eig[BadBCun1],' u(y=-1 )=0')
        print( '   ',evec[n-1,BadBCup1],'with associated eig-value of',eig[BadBCup1],' u(y=1 )=0')
        print( '  bad boundary condition eigenfunctions satisfying v=0')
        print( '   ',evec[n,BadBCvn1],'with associated eig-value of',eig[BadBCvn1],' v(y=-1 )=0')
        print( '   ',evec[2*n-1,BadBCvp1],'with associated eig-value of',eig[BadBCvp1],' v(y=1 )=0')
        print( '  bad boundary condition eigenfunctions satisfying w=0')
        print( '   ',evec[2*n,BadBCwn1],'with associated eig-value of',eig[BadBCwn1],' w(y=-1 )=0')
        print( '   ',evec[3*n-1,BadBCwp1],'with associated eig-value of',eig[BadBCwp1],' w(y=1 )=0')
        #print( '  bad boundary condition eigenfunctions satisfying P=0')
        #print( '   ',evec[3*n,BadBCpn1],'with associated eig-value of',eig[BadBCpn1],' p(y=-1 )=0')
        #print( '   ',evec[4*n-1,BadBCpp1],'with associated eig-value of',eig[BadBCpp1],' p(y=1 )=0')
        
        # now check u'=v'=w'=0 using forward and backward 1st diff 2nd order
        uy_bot = -3.*evec[0,:] + 4.*evec[1,:] - evec[2,:]
        vy_bot = -3.*evec[n,:] + 4.*evec[n+1,:] - evec[n+2,:]
        wy_bot = -3.*evec[2*n,:] + 4.*evec[2*n+1,:] - evec[2*n+2,:]
        # now at other wall
        uy_top = 1.*evec[n-3,:] -4.*evec[n-2,:] + 3.*evec[n-1,:]
        vy_top = 1.*evec[2*n-3,:] -4.*evec[2*n-2,:] + 3.*evec[2*n-1,:]
        wy_top = 1.*evec[3*n-3,:] -4.*evec[3*n-2,:] + 3.*evec[3*n-1,:]
        # plot derivative at walls for every eigenfunction for v=0
        fig=plt.figure(figsize=(5,4))
        plt.title('d/dy [u,v,w]=0 at wall check')
        plt.plot(uy_bot[eig_i],'o',label='u bot')
        plt.plot(uy_top[eig_i],'.',label='u top')
        plt.plot(vy_bot[eig_i],'o',label='v bot')
        plt.plot(vy_top[eig_i],'.',label='v top')
        plt.plot(wy_bot[eig_i],'o',label='w bot')
        plt.plot(wy_top[eig_i],'.',label='w top')
        plt.ylabel(r'$\frac{d\hat{v}}{dy}(y=\pm 1)$')
        plt.xlabel(r'$\omega_{max}$ to $\omega_{min}$')
        plt.legend(loc='best',numpoints=1,frameon=False)
        fig.tight_layout()

    if iplot:
        # plot Orr-Sommerfeld-Squire spectrum
        fig=plt.figure(figsize=(4,4))
        ax=plt.subplot(111)
        ax.plot(eig.real,eig.imag,'bo')
        ax.set_xlabel(r'$\omega_r$')
        ax.set_ylabel(r'$\omega_i$')
        ax.axis([0,1,-1.,0])
        ax.set_title('eigenvalues')
        fig.tight_layout()
        plt.show()
    return eig,evec,eig_i

In [ ]:
%%time 
eig_p,evec_p,eig_i_p = set_and_solve_eig_primitive(
    n=101,
    iBCs=False,
    iplot=True,
    alpha=1.,
    beta=0.,
    Re=10000.,
    order=2,
    large_multiple=6e1,)
eig,evec,eig_i = set_and_solve_eig(
    n=101,
    iBCs=False,
    iplot=True,
    alpha=1.,
    beta=0.,
    Re=10000.,
    order=2,
    large_multiple=6e1,)

## Solve using primitive variables, but while using $\nabla^2$ in each equation


In [ ]:
# functions for primitive formulation u,v,w,P
def set_BCs_primitive_lap(y, Aus, Bus, Avs, Bvs, Aws, Bws, Acons, Bcons, order=2,large_multiple=0.):
    '''
    Set boundary condition values to As, Bs
    Modifies the (n-2 X n) matrices to be (n X n)
    Sets first/last row of Aos, Bos to have the BC v'=0 at walls
    Adds first/last rows to all matrices to have v=eta=0 at walls
    
    Inputs:
        y: y spactial array 
        Aus: List of [Auu, Auv, Auw, Aup]
        Bus: List of [Buu, Buv, Buw, Bup]
        Avs: List of [Avu, Avv, Avw, Avp]
        Bvs: List of [Bvu, Bvv, Bvw, Bvp]
        Aws: List of [Awu, Awv, Aww, Awp]
        Bws: List of [Bwu, Bwv, Bww, Bwp]
        Acons: List of [Aconu, Aconv, Aconw, Aconp]
        Bcons: List of [Bconu, Bconv, Bconw, Bconp]
        order: order of derivative (O(h^order))
        large_multiple: used to set RHS to nonzero and kill spurious eigenmodes
    '''
    h=y[1]-y[0]
    # v'=0 at walls
    # alter first and last line of As and Bs to contain u'=v'=w'=0 BC
    d=1 # derivative
    N=order+d # how many pts needed for order of accuracy
    s = np.arange(N) # stencil for shifted diff of order for i=0
    sn = -1*np.arange(N) # stencil for shifted diff of order for i=-1
    Coeffs = get_D_Coeffs(s,d=d)
    Coeffsn = get_D_Coeffs(sn,d=d)
    # set u-mom u'=0 at walls
    for i in range(len(Aus)):
        Aus[i][(0,-1),:] = 0.
        Bus[i][(0,-1),:] = 0.
        if i==0:
            Aus[i][0,s] = Coeffs
            Bus[i][0,s] = large_multiple*Coeffs
            Aus[i][-1,sn-1] = Coeffsn
            Bus[i][-1,sn-1] = large_multiple*Coeffsn
    # set v-mom v'=0 at walls
    for i in range(len(Avs)):
        Avs[i][(0,-1),:] = 0.
        Bvs[i][(0,-1),:] = 0.
        if i==1:
            Avs[i][0,s] = Coeffs
            Bvs[i][0,s] = large_multiple*Coeffs
            Avs[i][-1,sn-1] = Coeffsn
            Bvs[i][-1,sn-1] = large_multiple*Coeffsn
            
    # set w-mom w'=0 at walls
    for i in range(len(Aws)):
        Aws[i][(0,-1),:] = 0.
        Bws[i][(0,-1),:] = 0.
        if i==2:
            Aws[i][0,s] = Coeffs
            Bws[i][0,s] = large_multiple*Coeffs
            Aws[i][-1,sn-1] = Coeffsn
            Bws[i][-1,sn-1] = large_multiple*Coeffsn
            
    # v=0 at walls
    # rows to add to As and Bs at top and bottom
    zero_row = np.zeros((1,Aus[0][0,:].size))
    v0_at_bottom = np.copy(zero_row)
    v0_at_bottom[0,0] = 1.
    v0_at_top = np.copy(zero_row)
    v0_at_top[0,-1] = 1.
    # add them to As and Bs
    # for Aus Bus u=0
    for i in range(len(Aus)):
        if i==0:
            Aus[i] = np.concatenate((v0_at_bottom,Aus[i],v0_at_top),axis=0)
            Bus[i] = np.concatenate((large_multiple*v0_at_bottom,Bus[i],large_multiple*v0_at_top),axis=0)
        else:
            Aus[i] = np.concatenate((zero_row,Aus[i],zero_row),axis=0)
            Bus[i] = np.concatenate((zero_row,Bus[i],zero_row),axis=0)
    # for Avs Bvs v=0
    for i in range(len(Avs)):
        if i==1:
            Avs[i] = np.concatenate((v0_at_bottom,Avs[i],v0_at_top),axis=0)
            Bvs[i] = np.concatenate((large_multiple*v0_at_bottom,Bvs[i],large_multiple*v0_at_top),axis=0)
        else:
            Avs[i] = np.concatenate((zero_row,Avs[i],zero_row),axis=0)
            Bvs[i] = np.concatenate((zero_row,Bvs[i],zero_row),axis=0)
    # for Aws Bws w=0
    for i in range(len(Aws)):
        if i==2:
            Aws[i] = np.concatenate((v0_at_bottom,Aws[i],v0_at_top),axis=0)
            Bws[i] = np.concatenate((large_multiple*v0_at_bottom,Bws[i],large_multiple*v0_at_top),axis=0)
        else:
            Aws[i] = np.concatenate((zero_row,Aws[i],zero_row),axis=0)
            Bws[i] = np.concatenate((zero_row,Bws[i],zero_row),axis=0)
    # for Acons Bcons P=0
    #for i in range(len(Acons)):
    #    if i==3:
    #        Acons[i] = np.concatenate((v0_at_bottom,Acons[i],v0_at_top),axis=0)
    #        Bcons[i] = np.concatenate((large_multiple*v0_at_bottom,Bcons[i],large_multiple*v0_at_top),axis=0)
    #    else:
    #        Acons[i] = np.concatenate((zero_row,Acons[i],zero_row),axis=0)
    #        Bcons[i] = np.concatenate((zero_row,Bcons[i],zero_row),axis=0)
    return (Aus,Bus,Avs, Bvs, Aws, Bws, Acons, Bcons)
def set_A_and_B_primitive_lap(y,alpha=1,beta=0.,Re=2000.,order=2,large_multiple=2.):
    '''
    Set A and B matrix for stated eigenvalue equation Aq=\omega Bq
    Input:
        y: array of y values of channel
        alpha=1 : alpha value for eigenvalue equation for channel flow
        Re=2000. : Reynolds number for flow
        order: order of accuracy of finite difference derivatives to use
    Output:
        A: matrix for LHS of eigenvalue equation
        B: matrix for RHS of eigenvalue equation
    '''
    #h=y[1]-y[0] # uniform spacing is assumed
    k2=alpha**2 + beta**2
    ialpha=1.j * alpha
    ibeta =1.j * beta
    
    # identity matrix
    I = np.eye(y.size)
    Im2=I[1:-1,:] # skip first and last rows (at walls)
    zero_mat=0.*Im2
    zero_mat_full = 0.*I
    # base flow
    U,Uy,Uyy = base_flow(y)
    
    # calculate derivatives
    D1 = set_D(y,order=order,d=1)
    D1_full = set_D(y,order=order,d=1,output_full=True)
    D2 = set_D(y,order=order,d=2)
    D3 = set_D(y,order=order,d=3)
    D4 = set_D(y,order=order,d=4)
    
    # for Orr-Sommerfeld eq. of the form
    # set constants and Au Bu
    Auu = ((ialpha*k2*U + ialpha*Uyy-1./Re * k2**2)*Im2
           + 2.*ialpha*Uy * D1
           + (1./Re*2.*k2 + ialpha*U)*D2
           - 1./Re * D4)
    Auv = ((Uy*k2+2.*Uyy)*Im2
           + Uy*D2)
    Auw = zero_mat
    Aup = (ialpha*k2*Im2
           + ialpha*D2)
    Aus = [Auu, Auv, Auw, Aup]
    Buu = (1.j*k2*Im2
           + 1.j*D2)
    Buv = zero_mat
    Buw = zero_mat
    Bup = zero_mat
    Bus = [Buu, Buv, Buw, Bup]
    
    # for constants and Av Bv
    Avu = zero_mat
    Avv = ((ialpha*U*k2 + ialpha*Uyy - 1./Re * k2**2)*Im2
           + (2.*ialpha*Uy*D1)
           + (ialpha*U + 1./Re * 2 * k2)*D2
           - 1./Re * D4)
    Avw = zero_mat
    Avp = (k2*D1 + D3)
    Avs = [Avu, Avv, Avw, Avp]
    Bvu = zero_mat
    Bvv = (1.j*k2 * Im2 
          + 1.j*D2)
    Bvw = zero_mat
    Bvp = zero_mat
    Bvs = [Bvu, Bvv, Bvw, Bvp]
    
    # for constants and Aw Bw
    Awu = zero_mat
    Awv = zero_mat
    Aww = ((ialpha*k2*U + ialpha*Uyy - 1./Re * k2**2)*Im2
          + 2.*ialpha*Uy*D1
          + (ialpha*U + 1./Re * k2)*D2
          - 1./Re * D4)
    Awp = ((ibeta*k2)*Im2
           + ibeta*D2)
    Aws = [Awu, Awv, Aww, Awp]
    Bwu = zero_mat
    Bwv = zero_mat
    Bww = (1.j*k2*Im2 
          + 1.j*D2)
    Bwp = zero_mat
    Bws = [Bwu, Bwv, Bww, Bwp]
    
    # for constants and Acon Bcon
    acon = ialpha
    bcon = 1.
    ccon = ibeta
    Aconu = acon*I #do full array, since no BCs to add
    Aconv = bcon*D1_full
    Aconw = ccon*I
    Aconp = zero_mat_full
    Acons = [Aconu, Aconv, Aconw, Aconp]
    Bconu = zero_mat_full
    Bconv = zero_mat_full
    Bconw = zero_mat_full
    Bconp = zero_mat_full
    Bcons = [Bconu, Bconv, Bconw, Bconp]
    
    # BCs
    Aus,Bus,Avs,Bvs,Aws,Bws,Acons,Bcons = set_BCs_primitive_lap(y, Aus, Bus, Avs, Bvs, Aws, Bws, Acons, Bcons, order=order,large_multiple=large_multiple)
    
    #combine to A and B for combined eqs.
    A = np.concatenate((
            np.concatenate(Aus,axis=1), # u-mom
            np.concatenate(Avs,axis=1), # v-mom
            np.concatenate(Aws,axis=1), # w-mom
            np.concatenate(Acons,axis=1))# continuity
        ,axis=0)
    B = np.concatenate((
            np.concatenate(Bus,axis=1), # u-mom
            np.concatenate(Bvs,axis=1), # v-mom
            np.concatenate(Bws,axis=1), # w-mom
            np.concatenate(Bcons,axis=1)) # continuity
        ,axis=0)
    
    return (A,B)
def set_and_solve_eig_primitive_lap(n=201,iBCs=True,iplot=True,**kwargs):
    '''
    Inputs:
        n=201:   number of pts in spatial y for finite difference scheme
        **kwargs: inputs for set_A_and_B and set_D and set_BCs
    Returns:
        eig:     eigenvalues of the equation
        evec:    eigenvectors of the equation
        eig_i:   index for eig and evec for max(eig) to smallest(eig) by using np.sort()
    '''
    print('inputs:')
    print('    n=%i'%(n),end='')
    for k in kwargs.items():
        print(k,end='')
    print('')
    
    # create y
    y = np.linspace(-1,1,n)
    # solve eigenvalue problem
    A,B = set_A_and_B_primitive_lap(y,**kwargs)
    eig,evec = linalg.eig(A,b=B)
    
    # sort in order of decending eigenvalues using argsort and print max,min,nans,inf
    eig_i = eig.argsort().imag[::-1]   
    if iBCs:
        print( 'check max and min eig')
        print( '   ',eig[eig!=np.inf][np.nanargmax(np.abs(eig[eig!=np.inf]))])
        print( '   ',eig[eig!=np.inf][np.nanargmin(np.abs(eig[eig!=np.inf]))])
        print( 'check isnan and isinf')
        print( '   ',eig[np.isnan(eig)])
        print( '   ',eig[np.isinf(eig)])

        # check BCs
        print('check BCs u=u_y=v=v_y=w=w_y=0')
        # check values at walls
        BadBCun1 = evec[0,:]!=0
        BadBCup1 = evec[n-1,:]!=0
        BadBCvn1 = evec[n,:]!=0
        BadBCvp1 = evec[2*n-1,:]!=0
        BadBCwn1 = evec[2*n,:]!=0
        BadBCwp1 = evec[3*n-1,:]!=0
        #BadBCpn1 = evec[3*n,:]!=0
        #BadBCpp1 = evec[4*n-1,:]!=0

        print( '  bad boundary condition eigenfunctions satisfying u=0')
        print( '   ',evec[0,BadBCun1],'with associated eig-value of',eig[BadBCun1],' u(y=-1 )=0')
        print( '   ',evec[n-1,BadBCup1],'with associated eig-value of',eig[BadBCup1],' u(y=1 )=0')
        print( '  bad boundary condition eigenfunctions satisfying v=0')
        print( '   ',evec[n,BadBCvn1],'with associated eig-value of',eig[BadBCvn1],' v(y=-1 )=0')
        print( '   ',evec[2*n-1,BadBCvp1],'with associated eig-value of',eig[BadBCvp1],' v(y=1 )=0')
        print( '  bad boundary condition eigenfunctions satisfying w=0')
        print( '   ',evec[2*n,BadBCwn1],'with associated eig-value of',eig[BadBCwn1],' w(y=-1 )=0')
        print( '   ',evec[3*n-1,BadBCwp1],'with associated eig-value of',eig[BadBCwp1],' w(y=1 )=0')
        #print( '  bad boundary condition eigenfunctions satisfying P=0')
        #print( '   ',evec[3*n,BadBCpn1],'with associated eig-value of',eig[BadBCpn1],' p(y=-1 )=0')
        #print( '   ',evec[4*n-1,BadBCpp1],'with associated eig-value of',eig[BadBCpp1],' p(y=1 )=0')
        
        # now check u'=v'=w'=0 using forward and backward 1st diff 2nd order
        uy_bot = -3.*evec[0,:] + 4.*evec[1,:] - evec[2,:]
        vy_bot = -3.*evec[n,:] + 4.*evec[n+1,:] - evec[n+2,:]
        wy_bot = -3.*evec[2*n,:] + 4.*evec[2*n+1,:] - evec[2*n+2,:]
        # now at other wall
        uy_top = 1.*evec[n-3,:] -4.*evec[n-2,:] + 3.*evec[n-1,:]
        vy_top = 1.*evec[2*n-3,:] -4.*evec[2*n-2,:] + 3.*evec[2*n-1,:]
        wy_top = 1.*evec[3*n-3,:] -4.*evec[3*n-2,:] + 3.*evec[3*n-1,:]
        # plot derivative at walls for every eigenfunction for v=0
        fig=plt.figure(figsize=(5,4))
        plt.title('d/dy [u,v,w]=0 at wall check')
        plt.plot(uy_bot[eig_i],'o',label='u bot')
        plt.plot(uy_top[eig_i],'.',label='u top')
        plt.plot(vy_bot[eig_i],'o',label='v bot')
        plt.plot(vy_top[eig_i],'.',label='v top')
        plt.plot(wy_bot[eig_i],'o',label='w bot')
        plt.plot(wy_top[eig_i],'.',label='w top')
        plt.ylabel(r'$\frac{d\hat{v}}{dy}(y=\pm 1)$')
        plt.xlabel(r'$\omega_{max}$ to $\omega_{min}$')
        plt.legend(loc='best',numpoints=1,frameon=False)
        fig.tight_layout()

    if iplot:
        # plot Orr-Sommerfeld-Squire spectrum
        fig=plt.figure(figsize=(4,4))
        ax=plt.subplot(111)
        ax.plot(eig.real,eig.imag,'bo')
        ax.set_xlabel(r'$\omega_r$')
        ax.set_ylabel(r'$\omega_i$')
        ax.axis([0,1,-1.,0])
        ax.set_title('eigenvalues')
        fig.tight_layout()
        plt.show()
    return eig,evec,eig_i

In [ ]:
%%time
eig_p,evec_p,eig_i_p = set_and_solve_eig_primitive_lap(
    n=101,
    iBCs=False,
    iplot=True,
    alpha=1.,
    beta=0.,
    Re=10000.,
    order=4,
    large_multiple=6e1,)
eig_p,evec_p,eig_i_p = set_and_solve_eig_primitive_lap(
    n=101,
    iBCs=False,
    iplot=True,
    alpha=1.,
    beta=0.,
    Re=10000.,
    order=8,
    large_multiple=6e1,)
eig,evec,eig_i = set_and_solve_eig(
    n=101,
    iBCs=False,
    iplot=True,
    alpha=1.,
    beta=0.,
    Re=10000.,
    order=4,
    large_multiple=6e1,)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))